## Import library

In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

import yfinance as yf
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import sambo
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy

from backtesting.test import SMA, GOOG

# backtesting.set_bokeh_output(notebook=True)
            
import itertools 
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14080\1650128870.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
d:\Binance Tradebot\.venv\Lib\site-packages\backtesting\_plotting.py:53: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

## Import utils

In [2]:
from utils.loader import *
from utils.signals import *
from utils.trade import *
from utils.strategy import *
from utils.basic import *

## Plotting sample

In [3]:
# now = datetime.today().strftime('%Y-%m-%d')

loader = DataLoader(ticker='AAPL', start='2012-01-01', end='2019-12-31', freq='1d', test_size=0.2)
loader.run()
# test     

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


## RSI strategy evaluation 

In [4]:
# strategy = BarUpDnStrategy
strategy = BollingerBandsStrategy
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.3}
bt.evaluate(data=bt.train_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
# bt.trades.head()

Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hell

Start                     2012-01-03 00:00:00
End                       2018-05-23 00:00:00
Duration                   2332 days 00:00:00
Exposure Time [%]                     24.4403
Equity Final [$]                  11637.92169
Equity Peak [$]                   12168.66477
Commissions [$]                    1106.76974
Return [%]                           16.37922
Buy & Hold Return [%]               224.77728
Return (Ann.) [%]                     2.40561
Volatility (Ann.) [%]                14.44196
CAGR [%]                              1.65263
Sharpe Ratio                          0.16657
Sortino Ratio                         0.23786
Calmar Ratio                          0.07582
Max. Drawdown [%]                    -31.7271
Avg. Drawdown [%]                     -7.5916
Max. Drawdown Duration     2120 days 00:00:00
Avg. Drawdown Duration      368 days 00:00:00
# Trades                                   31
Win Rate [%]                         35.48387
Best Trade [%]                    

## SMA strategy evaluation

In [ ]:
class EMACross_test(Strategy):

    short_duration = 5  # Default values, can be overridden
    long_duration = 10

    id = 0
    take_profit_ratio = 1.3
    stop_loss_ratio = 0.9

    stop_loss_duration = 5

    lookback = 3
    atr_multiplier = 1

    def init(self):
        price = self.data.Close
        self.ma1 = self.I(EMA, price, self.short_duration, plot=True, overlay=True)
        self.ma2 = self.I(EMA, price, self.long_duration, plot=True, overlay=True)
        self.bbw = self.I(BBW, price, plot=False, overlay=False)
        # self.atr = self.I(ATR, price, plot=True, overlay=False)
        self.ma200 = self.I(EMA, price, 100)

        self.atr = self.I(ATR, self.data)
        self.previous_low = self.I(previous_low, self.data.Low, self.stop_loss_duration, plot = False)
        self.previous_high = self.I(previous_high, self.data.High, self.stop_loss_duration, plot = False)

    def next(self):
        entry_price = self.data.Close[-1]

        long_stop_loss = min(self.previous_low[-1], entry_price - self.atr_multiplier*self.atr[-1])
        short_stop_loss = max(self.previous_high[-1], entry_price + self.atr_multiplier*self.atr[-1])

        for trade in self.trades:
            if trade.tag == "Long":
                trade.sl = long_stop_loss
            elif trade.tag == "Short":
                trade.sl = short_stop_loss


        past_max = max(self.bbw[-self.lookback-1:-1])  # Maximum in last 5 periods
        current_bbw = self.bbw[-1]

        count_green = sum(self.data.Color[-2:])

        if crossover(self.ma1, self.ma2) and count_green >= 2 and current_bbw >= past_max:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long')
            
            for trade in self.trades:
                if trade.tag == 'Short':
                    trade.close()

        if crossover(self.ma2, self.ma1) and count_green == 0 and current_bbw >= past_max:
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Short')
            
            for trade in self.trades:
                if trade.tag == 'Long':
                    trade.close()


        # elif crossover(self.ma2, self.ma1):
        #     for trade in self.trades:
        #         if trade.tag ==f'Long {self.id-1}':
        #             trade.close()

strategy = EMACross_test
# strategy = BollingerBound
bt = BackTrader(data=loader.data)

# params = {
#     'short_duration': 5,
#     'long_duration': 10
# }
params = {'take_profit_ratio': 1.3}
bt.evaluate(data=bt.train_data, strategy=strategy, params=params, order_size=0.9999, plot=True)
# bt.trades.head()

## Cross-validation sample

In [ ]:
# grid_search = {
#     'short_duration': range(2, 4),
#     'long_duration': range(5, 11)
# }
grid_search={'take_profit_ratio': [1.3], 'stop_loss_ratio': [0.9]}

bt.cross_val(strategy=strategy, train_size=240, test_size=240, step_size=240, order_size=0.999, commission=0.002, grid=grid_search)

{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10000.0
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
9366.580567796404
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
10064.102733125912
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
9486.098580446507
{'take_profit_ratio': 1.3, 'stop_loss_ratio': 0.9}
12132.430188524131


10209.84241397859

In [ ]:
class MacdCross(Strategy):

    id = 0
    take_profit_ratio = 1.02
    stop_loss_ratio = 0.95

    def init(self):
        price = self.data.Close
        self.macd_line = self.I(MACD, price)[0]
        self.signal_line = self.I(MACD, price)[1]

    def next(self):
        entry_price = self.data.Close[-1]

        if crossover(self.macd_line, self.signal_line):
            self.buy(size = self.order_size, tp=entry_price*self.take_profit_ratio, sl=entry_price*self.stop_loss_ratio, tag=f'Long {self.id}')
            self.id += 1

        elif crossover(self.signal_line, self.macd_line):
            for trade in self.trades:
                if trade.tag ==f'Long {self.id-1}':
                    trade.close()